In [1]:
import pandas as pd 
import numpy as np
import time

In [2]:
df_exrate = pd.read_csv("SimExRatesFall19.csv")
df_exrate.loc[:, "Unhedge_rev"] = df_exrate.apply(lambda x : (645 * 0.6513 * (1 + x["DM"] / 100) + 272 * 1.234 * (1 + x["BP"] / 100)), axis = 1)
df_exrate.loc[:, "future_DM"] = df_exrate['DM'].apply(lambda x : 0.6513 * (1 + x / 100))
df_exrate.loc[:, "future_BP"] = df_exrate['BP'].apply(lambda x : 1.234 * (1 + x / 100))

In [3]:
df_put = pd.DataFrame({"Price_DM" : [0.66,0.65,0.64,0.63,0.62,0.61,0.60,0.59,0.55],
                       "Cost_DM" : [0.085855,0.032191,0.020795,0.017001,0.013711,0.010851,0.008388,0.006291,0.001401], 
                       "Price_BP" : [1.3,1.25,1.20,1.15,1.1,1.05,1,0.95,0.9], 
                       "Cost_BP" : [0.137213,0.082645,0.0450460,0.028348,0.016146,
0.007860,0.003277,0.001134,0.000245]})

In [4]:
df_cal_DM = pd.DataFrame()
for i in df_exrate['future_DM']:
    tmp_ = df_put['Price_DM'] - i

    tmp_ = tmp_.apply(lambda x : max(x, 0))
    tmp_ = tmp_ - df_put.loc[:, "Cost_DM"] * 500
    df_cal_DM.loc[:, str(i)] = tmp_
    
df_cal_BP = pd.DataFrame()
for i in df_exrate['future_BP']:
    tmp_ = df_put['Price_BP'] - i
    tmp_ = tmp_.apply(lambda x : max(x, 0))
    tmp_ = tmp_ - df_put.loc[:, "Cost_BP"] * 500
    df_cal_BP.loc[:, str(i)] = tmp_

In [5]:
def InitialSolution(stepsize = 3):
    dm = np.random.dirichlet(np.ones(9),size=1)[0]
    bp = np.random.dirichlet(np.ones(9),size=1)[0]
    stepsize = np.full((18), 3)
    return np.concatenate((dm, bp)), stepsize

def getPopulations(population_size = 1000):
    populations = [InitialSolution() for i in range(population_size)]
    return populations

def getChild(populations, population_size = 1000):
    parents_idx = np.random.randint(0,population_size, size = 2)
    parents = populations[parents_idx[0]], populations[parents_idx[1]]
    child_fst = [parents[np.random.randint(0, 2, 1)[0]][0][i] for i in range(18)]

    child_sec = (parents[0][1] + parents[1][1]) * 0.5

    rand_norm = np.random.normal(0, 1, 1)
    mutation = [(child_sec[i] * np.exp(rand_norm + np.random.normal(0, 1, 1)))[0] for i in range(18)]

    child_fst = [np.maximum((child_fst[i] + np.random.normal(0, mutation[i], 1)), 0)[0] for i in range(18)]

    return np.array(child_fst), np.array(mutation)

def normalize(child):
    dm = np.array(child[0][0:9])
    if sum(dm) == 0:
        dm = dm
    else :
        dm = dm / sum(dm)
    bp = np.array(child[0][9:18])
    if sum(bp) == 0:
        bp = bp
    else:
        bp = bp / sum(bp)
    return np.concatenate((dm, bp)), child[1]
    
def calRev(df_cal_DM, df_cal_BP, normalize_child, future_DM, future_BP, unhedge_rev):
    rev = sum(df_cal_DM.loc[:, str(future_DM)].values * normalize_child[0:9]) + sum(df_cal_BP.loc[:, str(future_BP)].values * normalize_child[9:18]) + unhedge_rev
    return rev

def simRev(normalize_child, df_exrate):
    rev = [calRev(df_cal_DM, df_cal_BP, normalize_child, i[0], i[1], i[2]) for i in df_exrate[['future_DM', 'future_BP', 'Unhedge_rev']].values]
    return np.array(rev).mean()



def simProb(normalize_child, df_exrate):
    rev = np.array([calRev(df_cal_DM, df_cal_BP, normalize_child, i[0], i[1], i[2]) for i in df_exrate[['future_DM', 'future_BP', 'Unhedge_rev']].values])
    prob = sum(rev < 706) / 1000
    return prob
    
    


    
    

# 1.max rev

In [25]:
dict_sol = {}
best_list = []
gen_list = []
for i in range(10):
    MAXGEN = 1000
    POPULATION_SIZE = 10
    CHILDREN_SIZE = 50
    MAX_TIMES_NOT_IMPROVED = 10
    times_not_improved = 0
    best = 0
    time_cal_rev = 0
    best_gen = 0

    for gen in range(MAXGEN):
        if gen == 0:
            population = getPopulations(POPULATION_SIZE)
        children = [normalize(getChild(population, POPULATION_SIZE)) for i in range(CHILDREN_SIZE)]
        df_rev = pd.DataFrame(children)
        df_rev.loc[:, "rev"] = [simRev(i, df_exrate) for i in df_rev[0].values]
        idx = df_rev.sort_values("rev", ascending = False).index[0:POPULATION_SIZE]
        tmp_best = df_rev.sort_values("rev", ascending=False)["rev"].iloc[0]
        population = df_rev.loc[idx, [0, 1]].values.tolist()
        print(f'Best value in {gen} is {tmp_best}')
        print(df_rev.sort_values("rev", ascending=False).iloc[0][0])
        
        if tmp_best > best:
            best = tmp_best
            times_not_improved = 0
            tmp_best_sol = df_rev.sort_values("rev", ascending=False)[0].iloc[0]
            best_gen = gen
        else:
            times_not_improved += 1
            print(f'Not improved at :{best}, generation : {gen}')
            if times_not_improved == MAX_TIMES_NOT_IMPROVED:
                print(f'Optimal:{best} at {gen}')
                break
    dict_sol[i] = tmp_best_sol
    best_list.append(best)
    gen_list.append(best_gen)
    
        
        
        
        
        
        

Best value in 0 is 750.3111561739082
[0.         0.         0.19272589 0.0219257  0.24103709 0.00422765
 0.         0.20828818 0.33179549 0.         0.05784776 0.
 0.         0.05195154 0.         0.78467262 0.         0.10552808]
Best value in 1 is 754.3402684423314
[0.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.06097205 0.93902795 0.        ]
Best value in 2 is 752.3028397706731
[0.         0.09855192 0.         0.         0.01928574 0.01149864
 0.85162273 0.         0.01904097 0.         0.         0.
 0.09809539 0.         0.         0.         0.         0.90190461]
Not improved at :754.3402684423314, generation : 2
Best value in 3 is 754.3046265082953
[0.02150542 0.00193162 0.02669623 0.         0.02283512 0.47170746
 0.         0.         0.45532415 0.         0.         0.
 0.         0.         0.00748783 0.00249443 0.99001774 0.        ]
Not improved at :754.340268442331

Best value in 26 is 758.2878212765597
[8.44071592e-06 0.00000000e+00 5.29441911e-06 0.00000000e+00
 2.23995964e-06 0.00000000e+00 0.00000000e+00 9.99344854e-04
 9.98984680e-01 0.00000000e+00 3.16747104e-05 0.00000000e+00
 1.27017230e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.92430049e-02 9.00598303e-01]
Not improved at :758.5811062626357, generation : 26
Best value in 27 is 758.308228917283
[5.79846605e-05 0.00000000e+00 4.10165055e-05 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.90596988e-04
 9.99410402e-01 0.00000000e+00 4.32246814e-04 0.00000000e+00
 1.52187314e-05 2.69717304e-04 0.00000000e+00 3.57958992e-03
 0.00000000e+00 9.95703227e-01]
Not improved at :758.5811062626357, generation : 27
Best value in 28 is 758.3113530217495
[1.21590116e-08 0.00000000e+00 5.91567708e-05 4.74869657e-07
 1.53259831e-05 0.00000000e+00 0.00000000e+00 5.54062079e-05
 9.99869624e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 3.23283984e-03 0.00000000e+00 0.

# 2. min prob

In [ ]:
dict_prob = {}
best_prob_list = []
gen_prob_list = []


for i in range(10):
    MAXGEN = 1000
    POPULATION_SIZE = 10
    CHILDREN_SIZE = 50
    MAX_TIMES_NOT_IMPROVED = 10
    times_not_improved = 0
    best = 1
    time_cal_prob = 0
    best_gen_prob = 0

    for gen in range(MAXGEN):
        if gen == 0:
            population = getPopulations(POPULATION_SIZE)
        children = [normalize(getChild(population, POPULATION_SIZE)) for i in range(CHILDREN_SIZE)]
        df_prob = pd.DataFrame(children)
        st_cal_prob = time.time()
    #     df_prob.loc[:, "prob"] = df_prob.apply(lambda x : simProb(x, df_exrate), axis = 1)
        df_prob.loc[:, "prob"] = [simProb(i, df_exrate) for i in df_prob[0].values]
        end_cal_prob = time.time()
        time_cal_prob += end_cal_prob - st_cal_prob
        print(time_cal_prob)
        idx = df_prob.sort_values("prob").index[0:POPULATION_SIZE]
        tmp_best = df_prob.sort_values("prob")["prob"].iloc[0]
        population = df_prob.loc[idx, [0, 1]].values.tolist()
        print(f'Best value in {gen} is {tmp_best}')
        print(df_prob.sort_values("prob").iloc[0][0])

        if tmp_best < best:
            best   = tmp_best
            times_not_improved = 0
            tmp_best_prob = df_prob.sort_values("prob")[0].iloc[0]
            best_gen_prob = gen
        else:
            times_not_improved += 1
            print(f'Not improved at :{best}, generation : {gen}')
            if times_not_improved == MAX_TIMES_NOT_IMPROVED:
                print(f'Optimal:{best} at {gen}')
                break
                
    dict_prob[i] = tmp_best_prob
    best_prob_list.append(best)
    gen_prob_list.append(best_gen_prob)